Import necessary packages

In [1]:
import anthropic
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_openai import ChatOpenAI
from pathlib import Path
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticToolsParser, PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from typing import Sequence, List
from langgraph.graph import MessageGraph, END
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_anthropic import ChatAnthropic
import nest_asyncio
nest_asyncio.apply()


#function to call a path to the file and read it. 
def context_gen(file_name):
    Folder = "Context_files"
    here = Path(locals().get('__file__', Folder)).resolve()
    parameter = (here / file_name).read_text()
    return parameter

#remove code fences from the output
def remove_code_fences(text):
    lines = text.split("\n")
    lines = [line for line in lines if not line.strip().startswith('```')]
    lines[0] = lines[0].replace(' -', '-', 1)
    print()
    return "\n".join(lines)

def wrap_at_chars(text, char, width):
    """Wraps text only at characters in the provided list (wrap_chars).

    Args:
        text: The text string to wrap.
        width: Maximum text width.

    Returns:
        A string with special marker (e.g. ' || ') inserted at wrap points.
    """
    segments = text.split(char)
    import textwrap

    wrapped_segments = [textwrap.fill(segment, width) for segment in segments]
    return wrapped_segments
#create inputs to the model, telling it what needs to be done. 
#provides system message
system1 = context_gen("system1.txt")

#provides the few shot examples
output_examples = context_gen("outputex.txt")

#provides the input examples
input_examples = context_gen("inputex.txt")

#provides the database schema
schema = context_gen("dataBaseSchema.txt")

refsystem = context_gen("ref_system copy.txt")

system3 = context_gen("system3.txt")

system2 = context_gen("system2.txt")

rag_prompt = context_gen("rag_prompt.txt")


#First input to the model, breaks query down and provides geometry definition

class analyzed_query(BaseModel):
    """Identifying whether a geometric feature is present in the database and stating the name of the feature"""
    answer: str= Field(description="YES or NO")
    features: List= Field(description="Name of the identified geometrical feature")

class Response(BaseModel):
    """Listing the answer of whether the feature is present in the database and the name of the feature"""
    Response: List[analyzed_query]

chat = ChatAnthropic(model_name="claude-3-haiku-20240307") 
#chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)



examples = [
    {"input": input_examples, "output": output_examples},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


In [2]:
'''change the human variable to test different inputs'''
human = ("Select all boltholes")


parser = PydanticOutputParser(pydantic_object=analyzed_query)
prompt = ChatPromptTemplate.from_messages([("system", rag_prompt + "Answer the user query. Print the output in Json format according to the instructions.\n{format_instructions}",), ("human", human),]).partial(format_instructions=parser.get_format_instructions())
chain = prompt | chat | parser
answers = []
while len(answers) < 5:
    feature_extractor = chain.invoke({})
    answer = feature_extractor.answer
    extracted_features = feature_extractor.features

    answers.append(feature_extractor)

print(answers)

for item in answers:
    yes_list = []
    no_list = []
    if item in answers == answers[1]:
        yes_list.append(feature_extractor.features)
    else:
        no_list.append(feature_extractor.features)
        
print(len(yes_list), len(no_list))
    

[analyzed_query(answer='NO', features=['boltholes']), analyzed_query(answer='NO', features=['boltholes']), analyzed_query(answer='NO', features=['boltholes']), analyzed_query(answer='NO', features=['boltholes']), analyzed_query(answer='NO', features=['boltholes'])]
0 1


In [3]:
loader = DirectoryLoader("RAG")
raw_docs = loader.load()
textsplitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
docs = textsplitter.split_documents(raw_docs)
db = FAISS.from_documents(docs, OpenAIEmbeddings())
retriever = db.as_retriever()
 

prompt_for_rag = ChatPromptTemplate.from_template("""You are an assistant that is an expert at RAG (Retrieval Augmented Generation). You have been tasked with looking for the definition of the word provided to you as input and generate a summary of the available information."
            You will use only the context provided to you.                                 
            Context:{context}
            Query_to_db:{query}""")
rag_chain = ({"context": retriever, "query": RunnablePassthrough()}) | prompt_for_rag | chat | StrOutputParser()
rag_output = rag_chain.invoke(f"What is the definition of {extracted_features}?")
print(rag_output)

Based on the provided context, the definition of boltholes is:

Boltholes are cylindrical features used for fastening components together. In the edges table, boltholes can be identified by circular or elliptical edge types with small radii and relatively short lengths, representing the cylindrical holes for bolts.


In [4]:
#Base prompt chain, generates the first yaml version with minimal context

base_prompt = ChatPromptTemplate.from_messages([("system", system1), few_shot_prompt, MessagesPlaceholder(variable_name="messages")])
yaml_generator = base_prompt | chat 


def chain_yes():
    chain = yaml_generator | StrOutputParser()
    yaml = chain.invoke({"messages": [human]}, {"tags": ["chain_yes"]})
    print("RAG not required, generating YAML")
    print()
    print(yaml)
    print()
    return yaml

def chain_no():
    chain = yaml_generator | StrOutputParser()
    ai = AIMessage(content=rag_output)
    yaml = chain.invoke({"messages":[ai]}, {"tags": ["chain_no"]})
    print("Getting more information using RAG, then generating YAML")
    print()
    print(yaml)
    print()
    return yaml

def decision_maker(response_obj: analyzed_query):
    for item in response_obj:
        if item == "YES":
            return chain_yes()
        elif item == "NO":
            return chain_no()


In [5]:
model_output = []
for answer in feature_extractor:
    model_output.append(decision_maker(answer))

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'messages: roles must alternate between "user" and "assistant", but found multiple "assistant" roles in a row'}}

In [6]:
reflection_prompt = ChatPromptTemplate.from_messages([("system", refsystem), few_shot_prompt, ("ai", f"Here is some added context {rag_output}"), MessagesPlaceholder(variable_name="messages")])
reflect = reflection_prompt | chat | StrOutputParser()
feedback = reflect.invoke({"messages": [f"This is the YAML code you are supposed to provide feedback for: {model_output[0]}"]}, {"tags": ["feedback"]})

print(feedback)

wrapped_feedback = wrap_at_chars(feedback, ".", 80)
wrapped_ragoutput = wrap_at_chars(rag_output, ".", 80)

#Chain that takes the feedback into consideration and regenerates the YAML code that is then passed on to Simcenter
final_generate = base_prompt | chat | StrOutputParser()
output_yaml = final_generate.invoke({"messages": [f"I have some YAML code \n {model_output[0]}" + f"This code needs to be regenerated with the following instructions in mind: \n{feedback}"]}, {"tags": ["final_generate"]}) 

print(final_generate)



IndexError: list index out of range